# emon-handler-sandbox
Testing boto3 for use in a Lambda function.

In [1]:
import boto3
import time
import random
from decimal import Decimal

In [2]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('test-sensor-data')

In [5]:
# Create some mock event data
print('Creating mock data')
event_data = []
for i in range(5):
    msg = {
        'time': int(time.time()),
        'data': {
            'voltage': round(120 + random.uniform(-1, 1), 1),
            'current': 50 + random.randint(-40, 40)
        }
    }
    
    event_data.append(msg)
    print(f'Message {i+1}: {msg}')
    time.sleep(1)

print('\nWriting to DynamoDB...', end=' ')
    
with table.batch_writer() as batch:
    for msg in event_data:
        item = {
            'timestamp': msg['time'],
            'voltage': Decimal(str(msg['data']['voltage'])),
            'current': msg['data']['current'],
            'power': int(msg['data']['voltage'] * msg['data']['current']),
            'expdate': msg['time'] + 604800
        }
        
        batch.put_item(Item=item)
        
print('Done!')

Creating mock data
Message 1: {'time': 1641671208, 'data': {'voltage': 119.3, 'current': 71}}
Message 2: {'time': 1641671209, 'data': {'voltage': 120.9, 'current': 50}}
Message 3: {'time': 1641671210, 'data': {'voltage': 119.5, 'current': 34}}
Message 4: {'time': 1641671211, 'data': {'voltage': 119.0, 'current': 44}}
Message 5: {'time': 1641671212, 'data': {'voltage': 120.6, 'current': 58}}

Writing to DynamoDB... Done!
